<a href="https://colab.research.google.com/github/Andriusma1989/DL_keras-pytorch/blob/main/DL_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [131]:
#loading and tranasforming data
df = pd.read_csv("/content/drive/MyDrive/CA/DL homework/Cancer_Data.csv")
df = df.drop(columns=['Unnamed: 32', 'id'])

#  encode the diagnosis column as a binary variable
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Pytorch model

In [123]:
# Converting the data to PyTorch tensors
X_train, y_train = torch.tensor(X_train).float(), torch.tensor(y_train.values).float()
X_test, y_test = torch.tensor(X_test).float(), torch.tensor(y_test.values).float()

# Creating PyTorch datasets and data loaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Defining the neural network model
class CancerClassifier(nn.Module):
    def __init__(self):
        super(CancerClassifier, self).__init__()
        self.fc1 = nn.Linear(30, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Setting up the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Creating the model and moving it to the device
model = CancerClassifier().to(device)

# Setting up the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 50
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zeroing the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        # Calculating the loss
        loss = criterion(outputs.squeeze(), labels)
        # Backward pass
        loss.backward()
        # Updating the weights
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# Evaluating the model's performance on the testing set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Getting predictions
        outputs = model(inputs)
        # Converting predictions to binary labels
        predicted = (outputs.squeeze() > 0.5).float()
         # Counting total samples and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculating test accuracy
test_accuracy = correct / total
print('Test accuracy:', test_accuracy)

Epoch [1/50], Loss: 0.6415688395500183
Epoch [2/50], Loss: 0.6970514059066772
Epoch [3/50], Loss: 0.6035193800926208
Epoch [4/50], Loss: 0.389403373003006
Epoch [5/50], Loss: 0.31336289644241333
Epoch [6/50], Loss: 0.28075721859931946
Epoch [7/50], Loss: 0.31095975637435913
Epoch [8/50], Loss: 0.10873454064130783
Epoch [9/50], Loss: 0.16292020678520203
Epoch [10/50], Loss: 0.14896537363529205
Epoch [11/50], Loss: 0.03668161481618881
Epoch [12/50], Loss: 0.06928060948848724
Epoch [13/50], Loss: 0.054217834025621414
Epoch [14/50], Loss: 0.10972851514816284
Epoch [15/50], Loss: 0.136240154504776
Epoch [16/50], Loss: 0.06411115825176239
Epoch [17/50], Loss: 0.17177827656269073
Epoch [18/50], Loss: 0.16658994555473328
Epoch [19/50], Loss: 0.02679847739636898
Epoch [20/50], Loss: 0.17363832890987396
Epoch [21/50], Loss: 0.003820141777396202
Epoch [22/50], Loss: 0.07359325885772705
Epoch [23/50], Loss: 0.426961749792099
Epoch [24/50], Loss: 0.012338193133473396
Epoch [25/50], Loss: 0.03271332

# Keras model





In [132]:
# define the neural network model using Keras
model = Sequential()
model.add(Dense(16, input_dim=30, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# training the model
# model.fit(X_train, y_train, epochs=50, batch_size=32)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# evaluate the model's performance on the testing
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
'''
Test loss: 0.0650860145688057
Test accuracy: 0.9824561476707458
'''

Epoch 1/50
12/12 [==============================] - 1s 19ms/step - loss: 0.6801 - accuracy: 0.5659 - val_loss: 0.6397 - val_accuracy: 0.7363
Epoch 2/50
12/12 [==============================] - 0s 6ms/step - loss: 0.5803 - accuracy: 0.8022 - val_loss: 0.5425 - val_accuracy: 0.8022
Epoch 3/50
12/12 [==============================] - 0s 5ms/step - loss: 0.4879 - accuracy: 0.8681 - val_loss: 0.4530 - val_accuracy: 0.8791
Epoch 4/50
12/12 [==============================] - 0s 7ms/step - loss: 0.4086 - accuracy: 0.9011 - val_loss: 0.3771 - val_accuracy: 0.8901
Epoch 5/50
12/12 [==============================] - 0s 7ms/step - loss: 0.3420 - accuracy: 0.9176 - val_loss: 0.3241 - val_accuracy: 0.9011
Epoch 6/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2915 - accuracy: 0.9258 - val_loss: 0.2862 - val_accuracy: 0.9011
Epoch 7/50
12/12 [==============================] - 0s 6ms/step - loss: 0.2515 - accuracy: 0.9313 - val_loss: 0.2575 - val_accuracy: 0.9011
Epoch 8/50
12/12 [=

'\nTest loss: 0.0650860145688057\nTest accuracy: 0.9824561476707458\n'